<a href="https://colab.research.google.com/github/SunetraGhosh/ML-projects/blob/main/pdfchatbot_mistral7B_with_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALLING REQUIRED LIBRARIES**

In [ ]:
!pip install --quiet transformers==4.36.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.6 MB/s eta 0:00:00


In [ ]:
!pip install --quiet -U accelerate

In [ ]:
!pip install --quiet torch

In [ ]:
!pip install --quiet transformers[numpy]

In [ ]:
!pip install --quiet transformers[sklearn]

In [ ]:
!pip install --quiet gradio gradio-pdf

In [ ]:
!pip install --quiet faiss-cpu  langchain pypdf tiktoken huggingface_hub sentence-transformers


In [ ]:
!apt-get install --quiet tesseract-ocr


Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!pip install --quiet InstructorEmbedding

In [ ]:
!pip install --quiet pandas pytesseract pdf2image opencv-python

In [ ]:
!pip install --quiet numpy

In [ ]:
!apt-get install --quiet -y poppler-utils

# Install pdf2image
!pip install --quiet pdf2image

# Import necessary libraries
from pdf2image import convert_from_path


Reading package lists...
Building dependency tree...
Reading state information...
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
from transformers import pipeline


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

**SETTING UP THE MODEL**

In [ ]:
!pip3 install --quiet optimum
#!pip3 install git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79
!pip3 install --quiet auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7


In [ ]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="cuda",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
embeddings=instructor_embeddings

llm = HuggingFacePipeline(pipeline=pipe)

**OCR**

In [ ]:
import cv2
import numpy as np

def deskew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    coords = np.column_stack(np.where(gray > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated


In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text


**BUILDING QA CHAIN**

In [ ]:
db=None

In [ ]:
def process_pdf_to_qa(pdf_file: str) -> str:
    file_path=pdf_file
    #loader = PyPDFLoader(pdf_file)
    #documents = loader.load()
    pages = convert_from_path(file_path)

    extracted_text = []

    for page in pages:
        # Step 2: Preprocess the image (deskew)
        preprocessed_image = deskew(np.array(page))

        # Step 3: Extract text using OCR
        text = extract_text_from_image(preprocessed_image)
        extracted_text.append(text)
    all_text = " ".join(extracted_text)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts = text_splitter.split_text(all_text)
    global db

    db = FAISS.from_texts(texts, embeddings)
    #retriever = db.as_retriever(search_kwargs={'k': 2})
    #qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever,return_source_documents=True)
    #bot_response = db.similarity_search(user_input)
    #return (bot_response[0].page_content)


    #return qa_chain["answer"];
#print(process_pdf_to_qa("/content/Only-Time-Will-Tell-PDF_231012_154449.pdf","who is the author of this book"))

In [ ]:
import time
from langchain.chains import RetrievalQA

In [ ]:
def respond(message, chat_history):
    retriever = db.as_retriever(search_kwargs={'k': 2})
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)
    result = qa_chain(message)
    chat_history.append((message, result['result']))
    time.sleep(2)
    return "", chat_history


**BUILDING GRADIO INTERFACE**

In [ ]:
import gradio as gr
from gradio_pdf import PDF


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown('# Q&A Bot for PDFs')
    with gr.Tab("Input Document"):
        gr.Interface(
    process_pdf_to_qa,
    [PDF(label="PDF Document")],outputs=gr.Textbox("proceed to next tab after bot is built"))

    with gr.Tab("Knowledge bot"):
      chatbot = gr.Chatbot()
      msg = gr.Textbox()
      clear = gr.Button("Clear")

      msg.submit(respond, [msg, chatbot], [msg, chatbot]
    )

      clear.click(lambda: None, None, chatbot, queue=False)


demo.queue().launch(debug = True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://21da5974b1a264477d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://21da5974b1a264477d.gradio.live


In [ ]:
!gradio deploy

Need 'write' access token to create a Spaces repo.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your t